<a href="https://colab.research.google.com/github/AlinaRodina99/diplom/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision

In [ ]:
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import pandas as pd
import os
from skimage import io
from PIL import Image, ImageFile
import numpy as np
import torch
import numpy as np

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
torch.manual_seed(17)

In [ ]:
class PaintingsAndPainters(Dataset):
  def __init__(self, csv_file, root_dir, transform):
    self.annotations = pd.read_csv(csv_file, header=None)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    y_label = torch.tensor(self.annotations.iloc[index, 1])
    img_path = self.root_dir + self.annotations.iloc[index, 0]
    image = Image.open(img_path).convert("RGB")
    #transform = transforms.Grayscale()
    if self.transform:
      image = self.transform(image)
    return (image, y_label)


In [ ]:
image_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

In [ ]:
dataset = PaintingsAndPainters(csv_file= 'data_and_labels_new.csv', root_dir = '/content/drive/MyDrive/images_1/', transform = image_transforms)

In [ ]:
m = len(dataset)

In [ ]:
m

2023

In [ ]:
from torch.utils.data import DataLoader,random_split
import torch

In [ ]:
def mean_std(loader):
  images, lebels = next(iter(loader))
  # shape of images = [b,c,w,h]
  mean, std = images.mean([0,2,3]), images.std([0,2,3])
  return mean, std

In [ ]:
new_image_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.4489, 0.3996, 0.3405], [0.2815, 0.2766, 0.2672])
    ])

In [ ]:
norm_dataset = PaintingsAndPainters(csv_file= 'data_and_labels.csv', root_dir = '/content/drive/MyDrive/images_init/', transform = new_image_transforms)

NameError: ignored

In [ ]:
generator_ = torch.Generator().manual_seed(17)
train_data, val_data = random_split(dataset, [int(m-m*0.1), int(m*0.1) + 1], generator = generator_)

In [ ]:
batch_size=8

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True, num_workers = 1)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle = False, num_workers = 1)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from torch import nn

In [ ]:
class Autoencoder(nn.Module):
  def __init__(self):
        super().__init__()

        self.decoder_conv = nn.Sequential(
            nn.Upsample(scale_factor = 2),
            nn.ReLU(),
            nn.ConvTranspose2d(200, 100, (5, 5)),
            nn.ReLU(),
            nn.Upsample(scale_factor = 2),
            nn.ReLU(),
            nn.ConvTranspose2d(100, 3, (5, 5))
        )

        ### Convolutional section
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(3, 100, 5),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.ReLU(),
            nn.Conv2d(100, 200, (5, 5)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
  def forward(self, x):
        x = self.encoder_conv(x)
        x = x.view(-1, 744200)
        x = self.decoder_conv(x)
        return x

In [ ]:
class CNN(Autoencoder):
  def __init__(self):
    super().__init__()

In [ ]:
loss_fn = torch.nn.MSELoss()

In [ ]:
lr = 0.01

In [ ]:
autoencoder = Autoencoder()
autoencoder.to(device)

Autoencoder(
  (decoder_conv): Sequential(
    (0): Upsample(scale_factor=2.0, mode='nearest')
    (1): ReLU()
    (2): ConvTranspose2d(200, 100, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Upsample(scale_factor=2.0, mode='nearest')
    (5): ReLU()
    (6): ConvTranspose2d(100, 3, kernel_size=(5, 5), stride=(1, 1))
  )
  (encoder_conv): Sequential(
    (0): Conv2d(3, 100, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(100, 200, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
)

In [ ]:
optim = torch.optim.Adam(autoencoder.parameters(), lr=lr, weight_decay = 1e-05)
val_loss_prev = 0

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/cnn_autoencoder.pt")
autoencoder.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
lr = checkpoint['lr']
counter = 1
lr

0.008337477621301497

In [ ]:
autoencoder.decoder_conv = nn.Sequential(
    nn.Linear(744200, 400),
    nn.ReLU(),
    nn.Linear(400, 200)
)

In [ ]:
autoencoder = autoencoder.to(device)

In [ ]:
def add_noise(inputs,noise_factor=0.2):
	noisy = inputs+torch.randn_like(inputs) * noise_factor
	noisy = torch.clip(noisy,0.,1.)
	return noisy

In [ ]:
def train_epoch(autoencoder, device, dataloader, loss_fn, optimizer, epoch):
    # Set train mode for both the encoder and the decoder
    train_loss = []
    autoencoder.train()
    avg_train_loss = 0
    count = 0
    global lr
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch, labels in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
      #noisy_data = add_noise(image_batch).to(device)
      #decoded_data = autoencoder(noisy_data)
      outputs = autoencoder(image_batch)
        # Evaluate loss
      loss = loss_fn(outputs, labels)
        # Backward pass
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        # Print batch loss
      print('\t partial train loss (single batch): %f' % (loss.data))
      avg_train_loss += loss.item() / len(dataloader)
      #torch.save({
            #'model_state_dict': autoencoder.state_dict(),
            #'optimizer_state_dict': optim.state_dict(),
            #'lr' : lr
            #}, "/content/drive/MyDrive/new_new_model.pt")
    
    lr *= 0.98
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=lr, weight_decay=1e-05)
    print('ccc')
    return avg_train_loss

In [ ]:
def test_epoch(autoencoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    autoencoder.eval()
    avg_val_loss = 0
    count = 0
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, labels in dataloader:
            # Move tensor to the proper device
            #if torch.cuda.is_available():
               #print('yes')
            image_batch.to(device)
            outputs = autoencoder(image_batch)
            # Decode data
            #decoded_data = autoencoder(image_batch)
            # Append the network output and the original image to the lists
            loss = loss_fn(outputs, labels)
            avg_val_loss += loss.item() / len(dataloader)
    return avg_val_loss

In [ ]:
val_losses = []
val_accs = []
def test_epoch(autoencoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    autoencoder.eval()
    avg_val_loss = 0
    predicted_labels = []
    real_labels = []
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, labels in dataloader:
            # Move tensor to the proper device
            #if torch.cuda.is_available():
               #print('yes')
            image_batch = image_batch.to(device)
            labels = labels.to(device)
            # Decode data
            outputs = autoencoder(image_batch)
            # Append the network output and the original image to the lists
            loss = loss_fn(outputs, labels)
            avg_val_loss += loss / len(dataloader)
            predicted_labels.append(outputs.argmax(dim = 1))
            real_labels.append(labels)
    predicted_labels = torch.cat(predicted_labels)
    real_labels = torch.cat(real_labels)
    val_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()
    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)
    return avg_val_loss, val_acc

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

In [ ]:
loss_fn = nn.CrossEntropyLoss()

dataset = ConcatDataset([train_data, val_data])

num_epochs=10
batch_size=128
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [ ]:

#for epoch in range(4, num_epochs):
#3
num_epochs = 30
for epoch in range(6, num_epochs):
      train_loss =train_epoch(autoencoder,device, train_loader,loss_fn, optim, epoch)
      val_loss = test_epoch(autoencoder, device,valid_loader,loss_fn)
      torch.save({
            'model_state_dict': autoencoder.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': val_loss,
            'counter': counter,
            #'scheduler': scheduler,
            'lr'
            
             : lr,
            'epoch': epoch
            }, "/content/drive/MyDrive/cnn_autoencoder.pt")
      #if epoch > 4:
      
        #val_loss_prev = checkpoint['loss']
        #if val_loss_prev - val_loss <= 0:
           #counter += 1
           #if counter > 5:
            # break
      print('\n EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))

	 partial train loss (single batch): 3.519453
	 partial train loss (single batch): 3.507554
	 partial train loss (single batch): 4.003544
	 partial train loss (single batch): 5.206771
	 partial train loss (single batch): 3.912924
	 partial train loss (single batch): 4.937459
	 partial train loss (single batch): 4.546063
	 partial train loss (single batch): 3.993280
	 partial train loss (single batch): 3.975260
	 partial train loss (single batch): 3.959958
	 partial train loss (single batch): 2.361553
	 partial train loss (single batch): 2.991948
	 partial train loss (single batch): 2.672703
	 partial train loss (single batch): 3.887312
	 partial train loss (single batch): 4.117747
	 partial train loss (single batch): 3.946783
	 partial train loss (single batch): 4.032081
	 partial train loss (single batch): 4.616119
	 partial train loss (single batch): 6.124329
	 partial train loss (single batch): 4.841477
	 partial train loss (single batch): 5.076682
	 partial train loss (single batch

TypeError: ignored

In [ ]:
num_epochs = 30
#for epoch in range(4, num_epochs):
#3
for epoch in range(4, num_epochs):
   train_loss =train_epoch(autoencoder,device, train_loader,loss_fn, optim, epoch)
   val_loss = test_epoch(autoencoder, device,valid_loader,loss_fn)
   torch.save({
            'model_state_dict': autoencoder.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': val_loss,
            'counter': counter,
            #'scheduler': scheduler,
            'lr' : lr,
            'epoch': epoch
            }, "/content/drive/MyDrive/model.pt")
   if epoch > 4:
      
     val_loss_prev = checkpoint['loss']
     if val_loss_prev - val_loss <= 0:
       counter += 1
       if counter > 5:
         break
   print('\n EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))

	 partial train loss (single batch): 0.051994
	 partial train loss (single batch): 0.106104
	 partial train loss (single batch): 0.072035
	 partial train loss (single batch): 0.074467
	 partial train loss (single batch): 0.062407
	 partial train loss (single batch): 0.069505
	 partial train loss (single batch): 0.080621
	 partial train loss (single batch): 0.062709
	 partial train loss (single batch): 0.111110
	 partial train loss (single batch): 0.081672
	 partial train loss (single batch): 0.075219
	 partial train loss (single batch): 0.091972
	 partial train loss (single batch): 0.091051
	 partial train loss (single batch): 0.068592
	 partial train loss (single batch): 0.063609
	 partial train loss (single batch): 0.076215
	 partial train loss (single batch): 0.071613
	 partial train loss (single batch): 0.053021
	 partial train loss (single batch): 0.077674
	 partial train loss (single batch): 0.089189
	 partial train loss (single batch): 0.066067
	 partial train loss (single batch